# Data Preprocessing Tools

## Importing the libraries

In [90]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Importing the dataset

In [91]:
# Your selected columns
s4_req_cols = ['SAP Personnel #', 'Current_Role', 'Birthdate', 'Street Address', 'City', 'State', 'Zip Code']
workday_req_cols = ['Employee_ID', 'Employee Annual Salary$', 'Monthly Medical contribution$', 'Monthly Dental Contribution$', 
                    'Monthly Vision Contribution$', 'Bonus $', 'Years_Of_Service', 'Last_Promoted', 'Department',
                    'Location', 'Gender', 'Employee HR rate', '# of Hours per week']

# Load only required columns
df_s4 = pd.read_excel("raw_data/resources_s4.xlsx", usecols=s4_req_cols)
df_workday = pd.read_excel("raw_data/resources_workday.xlsx", usecols=workday_req_cols)


In [92]:
print(df_s4.head())
print(df_workday.head())

   SAP Personnel #               Current_Role  Birthdate   Street Address  \
0               57   Senior Software Engineer 1991-05-19    1002 Maple Dr   
1               58   Associate Data Scientist 1996-03-23  1059 Taylor Ave   
2               59  Associate Product Manager 1993-09-25    1089 Adams St   
3               60           Business Analyst 1994-03-03     1095 Polk St   
4               61    Chief Operating Officer 1984-01-13     1021 Polk St   

            City State  Zip Code  
0  San Francisco    CA     94103  
1  San Francisco    CA     94111  
2       New York    NY     10009  
3        Chicago    IL     60605  
4         Boston    MA      2101  
   Employee_ID  Employee Annual Salary$  Monthly Medical contribution$  \
0           57                   130800                            130   
1           58                    74250                            105   
2           59                   125850                            130   
3           60                 

## Merging the dataset

In [93]:
# Merge on SAP Personnel # and Employee_ID
df_merged = pd.merge(df_s4, df_workday, left_on='SAP Personnel #', right_on='Employee_ID', how='inner')

# Drop both ID columns since they are just identifiers
df_merged.drop(columns=['SAP Personnel #', 'Employee_ID'], errors='ignore', inplace=True)

In [94]:
print(df_merged.head())

                Current_Role  Birthdate   Street Address           City State  \
0   Senior Software Engineer 1991-05-19    1002 Maple Dr  San Francisco    CA   
1   Associate Data Scientist 1996-03-23  1059 Taylor Ave  San Francisco    CA   
2  Associate Product Manager 1993-09-25    1089 Adams St       New York    NY   
3           Business Analyst 1994-03-03     1095 Polk St        Chicago    IL   
4    Chief Operating Officer 1984-01-13     1021 Polk St         Boston    MA   

   Zip Code  Employee Annual Salary$  Monthly Medical contribution$  \
0     94103                   130800                            130   
1     94111                    74250                            105   
2     10009                   125850                            130   
3     60605                    84750                            105   
4      2101                   300000                            230   

   Monthly Dental Contribution$  Monthly Vision Contribution$  Bonus $  \
0           

In [95]:
print(df_merged.shape)

(100, 18)


## Feature Engineering

In [96]:
df_merged['Last_Promoted'] = pd.to_datetime(df_merged['Last_Promoted'])
df_merged['Birthdate'] = pd.to_datetime(df_merged['Birthdate'])

today = pd.Timestamp.today()

# Calculate years since last promotion using days / 365.25
df_merged['Years_Since_Last_Promotion'] = (today - df_merged['Last_Promoted']).dt.days / 365.25

# Calculate age as integer years
df_merged['age'] = ((today - df_merged['Birthdate']).dt.days / 365.25).astype(int)


# Renaming DataFrame for feature-engineered data
df_featured = df_merged.copy()

# Optionally drop original columns
df_featured.drop(columns=['Last_Promoted', 'Birthdate'], inplace=True)

print(df_featured[['Years_Since_Last_Promotion', 'age']].head())

   Years_Since_Last_Promotion  age
0                    2.776181   34
1                    1.538672   29
2                    1.221081   31
3                    2.272416   31
4                    5.757700   41


In [97]:
print(df_featured.head())

                Current_Role   Street Address           City State  Zip Code  \
0   Senior Software Engineer    1002 Maple Dr  San Francisco    CA     94103   
1   Associate Data Scientist  1059 Taylor Ave  San Francisco    CA     94111   
2  Associate Product Manager    1089 Adams St       New York    NY     10009   
3           Business Analyst     1095 Polk St        Chicago    IL     60605   
4    Chief Operating Officer     1021 Polk St         Boston    MA      2101   

   Employee Annual Salary$  Monthly Medical contribution$  \
0                   130800                            130   
1                    74250                            105   
2                   125850                            130   
3                    84750                            105   
4                   300000                            230   

   Monthly Dental Contribution$  Monthly Vision Contribution$  Bonus $  \
0                            35                            19    13865   
1   

In [98]:
print(df_featured.columns)

Index(['Current_Role', 'Street Address', 'City', 'State', 'Zip Code',
       'Employee Annual Salary$', 'Monthly Medical contribution$',
       'Monthly Dental Contribution$', 'Monthly Vision Contribution$',
       'Bonus $', 'Years_Of_Service', 'Department', 'Location', 'Gender',
       'Employee HR rate', '# of Hours per week', 'Years_Since_Last_Promotion',
       'age'],
      dtype='object')


In [99]:
print(df_featured.shape)

(100, 18)


## Create Custom Dependent Column

In [100]:
# Create a boolean Series for condition
condition = df_featured['Years_Since_Last_Promotion'] > 5

# Group by that boolean condition
grouped = df_featured.groupby(condition)


In [101]:
print(grouped.size())

Years_Since_Last_Promotion
False    89
True     11
dtype: int64


In [102]:
df_featured['left'] = (df_featured['Years_Since_Last_Promotion'] > 5).astype(int)

# Renaming DataFrame for updated-engineered data
df_updated = df_featured.copy()

In [103]:
print(df_updated[['Years_Since_Last_Promotion', 'left']].head())

   Years_Since_Last_Promotion  left
0                    2.776181     0
1                    1.538672     0
2                    1.221081     0
3                    2.272416     0
4                    5.757700     1


## Export Updated Excel

In [104]:
# Create the folder if it doesn't exist
os.makedirs("updated_data", exist_ok=True)

# Save the Excel file inside the folder
df_updated.to_excel("updated_data/updated_data.xlsx", index=False)
